In [1]:
import sympy as sym
import math as m
import numpy as np
import pandas as pd

import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import SIC_POVM_functions as sic

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


[(6+0j), (6+0j), (2.4+0j), (2.5999999999999996+1.637849848627056e-17j), (2.287500258169+0j), (2.4+2.2311252215006176e-17j), (2.4-2.576281494458285e-17j), (2.3000000000000003-4.125283626344832e-18j), (2.587500258169+0j)]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:239: ComplexWarning: Casting complex values to real discards the imaginary part
  retval = _minpack._hybrd(func, x0, args, 1, xtol, maxfev,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [4]:
#Class definitions #1
class seed_sol:
    def __init__(self, seed):
        self.seed = seed
    @property
    def initial_guess(self):
        np.random.seed(self.seed)
        initial_guess = np.random.rand(35)  # random initial guess
        return initial_guess
    @property
    def solution(self):
        solution = fsolve(Lagrange_eqs, self.initial_guess)
        return solution
    @property
    def res_list(self):
        res_list = Lagrange_eqs(self.solution)
        return res_list
    @property
    def res(self):
        res = np.sum(np.abs(self.res_list))
        return res
    def full_POVM(self):
        #break solution into 4 parts, remove the last 7 elements
        sol = self.solution
        solution_add = [sol[:7], sol[7:14], sol[14:21], sol[21:28]]     # divide into 4 parts of 7
        POVM_normalized_9d = [list(i) for i in POVM_normalized_5d]         # deep copy 
        for i in range(2):                              # first two vecs are known
            POVM_normalized_9d[i].extend([0] *4)
        for part in solution_add:                       # adding (already normalized) elements from the solution to make the remaining vectors 9d
            for i in range(7):
                # POVM_normalized_9d[i+2].append(part[i])
                POVM_normalized_9d[i+2].append((1/np.sqrt(6))*(part[i]))        # normalizing coming from the fixing of the lagrangian eqns
        return POVM_normalized_9d


class POVM_relations:
    def __init__(self, POVM_vec_list):
        self.POVM_vec_list = np.array(POVM_vec_list)        # making array for easy dot product
        self.inner_products = {}
    def dot_product(self, i, j):
        if i < 0 or j < 0 or i > 8 or j > 8:
            raise ValueError('Bad index, must be integer 0 to 8')
        return np.vdot(self.POVM_vec_list[i], self.POVM_vec_list[j])
    def all(self, clean = True, threshold = 1e-12, absolutes = True):
        if len(self.POVM_vec_list) != 9:
            raise ValueError('The POVM list must have 9 elements')
        for i in range(9):
            for j in range(i, 9):
                key = f'{i+1}{j+1}'
                value = self.dot_product(i, j)
                self.inner_products[key] = value
        #cleaning the dictionary
        rel = self.inner_products
        for key in rel:
            if absolutes == True:
                rel[key] = abs(rel[key])
            if abs(rel[key]) < threshold:
                rel[key] = 0
        return rel


In [108]:
#Class definitions #2
class povm_create:                  # creates POVM object given a list of 28 coefficients (the unknown variables)
    def __init__(self, coeffs):     # coeffs is the given list of 28 coefficients, make sure it is unnormalized!
        self.coeffs = coeffs
    @property
    def full_POVM(self):
        #break solution into 4 parts, remove the last 7 elements
        sol = self.coeffs
        solution_add = [sol[:7], sol[7:14], sol[14:21], sol[21:28]]     # divide into 4 parts of 7
        POVM_normalized_9d = [list(i) for i in POVM_normalized_5d]         # deep copy 
        for i in range(2):                              # first two vecs are known
            POVM_normalized_9d[i].extend([0] *4)
        for part in solution_add:                       # adding (already normalized) elements from the solution to make the remaining vectors 9d
            for i in range(7):
                # POVM_normalized_9d[i+2].append(part[i])
                POVM_normalized_9d[i+2].append((1/np.sqrt(6))*(part[i]))        # normalizing coming from the fixing of the lagrangian eqns
        return POVM_normalized_9d



class POVM_gram_matrix:
    def __init__(self, POVM_vec_list):
        self.POVM_vec_list = np.array(POVM_vec_list)        # making array for easy dot product
    def all(self, clean=True, threshold=1e-12, absolutes=True):
        if len(self.POVM_vec_list) != 9:
            raise ValueError('The POVM list must have 9 elements')
        
        gram_matrix = np.zeros((9, 9))     # initialize 9x9 empty matrix
        for i in range(9):
            for j in range(i, 9):
                value = np.vdot(self.POVM_vec_list[i], self.POVM_vec_list[j])
                # Fill both [i, j] and [j, i] to maintain symmetry
                gram_matrix[i, j] = value
                gram_matrix[j, i] = np.conjugate(value)
        
        # Apply absolutes and threshold cleaning if needed
        if absolutes:
            gram_matrix = np.abs(gram_matrix)
        if clean:
            gram_matrix[gram_matrix < threshold] = 0
        return gram_matrix
    


In [109]:
# creating five lists POVMs

w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_unnormalized = [[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]             # unnormalized POVM direction vectors
# POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors
POVM_vec = (1/np.sqrt(2))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors

# c4j_list = [2,.5,.5,.5, (-.25-.433013j), -.25, .5, -.25, (-.25-.433013j)]           # fourth elements, not normalized yet. See sic.py for calcs
c4j_list = [2,.5,.5,.5, (-.25-.25j*np.sqrt(3)), (-.25+.25j*np.sqrt(3)), .5, (-.25+.25j*np.sqrt(3)), (-.25-.25j*np.sqrt(3))]           # fourth elements, not normalized yet. See sic.py for calcs
# c5j_list = [0, np.sqrt(15)/2 , 0.38729833462074165, (-0.38729833462074176-0.44721359549995776j), -0.19364916731037082, 0.5809475019311124, -0.38729833462074165, (-0.19364916731037093-0.4472135954999579j), 0.5809475019311124]
c5j_list = [0, np.sqrt(15)/2 , (1/np.sqrt(15))*(3/2), (1/np.sqrt(15))*(2*w**2-.5) ,-(1/np.sqrt(15))*(2+(5/2)*w**2), (1/np.sqrt(15))*(2-w/2), (1/np.sqrt(15))*(2*w-.5), -(1/np.sqrt(15))*(2+(5/2)*w), (1/np.sqrt(15))*(2-.5*w**2)]

POVM_unnormalized_5d = []
for i in range(len(POVM_unnormalized)):
	# Directly append c4j_list[i] and c5j_list[i] to the copies of lists in POVM_unnormalized
	vec_5d_i = POVM_unnormalized[i] + [c4j_list[i], c5j_list[i]]
	POVM_unnormalized_5d.append(vec_5d_i)

POVM_unnormalized_5d[0]    ## unnormalized 5-lists POVM vectors

POVM_normalized_5d = ((1/(6**.5))*(np.array(POVM_unnormalized_5d))).tolist()       # normalized 5-lists POVM vectors

#print and compare
# print(POVM_unnormalized_5d[0])	## UNnormalized 5-lists POVM vectors
# print(POVM_normalized_5d[0])	## normalized 5-lists POVM vectors

# delta_five  = [0.400000000000000, 0.433333333333333, 0.381250043028167, 0.400000000000000, 0.400000000000000, 0.383333333333333, 0.431250043028167]
# delta_five_new = [(0.40000000000000013+0j), (0.4333333333333334-1.5431352694421904e-18j), (0.3812500430281667+0j), (0.40000000000000013-3.0923901048776238e-18j), (0.4+1.898287466587076e-18j), (0.38333333333333347-7.881938927040631e-19j), (0.4312500430281667+3.469446951953614e-18j)]
# print('delta_five_new = ', [np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]) for i in range(9)])
delta_five_fixed = [ np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]) for i in range(2,9)]
print('delta_five_fixed = ', delta_five_fixed)


delta_five_fixed =  [(0.40000000000000013+0j), (0.4333333333333334-1.5431352694421904e-18j), (0.43333333333333335+1.0728477129103336e-18j), (0.43333333333333346-4.471148278649265e-19j), (0.4333333333333333+2.6275172237666522e-18j), (0.4333333333333335+2.267799535210488e-19j), (0.43333333333333335+6.251554030278836e-19j)]


### Using Conjugate gradient on residues (without using Lagrange method)

In [110]:
def Relation_Res(vars_full):     # vars should be a list of 56 elements - 28 real and corresponding to them 28 imaginary
    # if len(vars_full) != 28:
        # raise ValueError("Expected a list of 28 elements for vars.")
    # C63, C64, C65, C66, C67, C68, C69,  C73, C74, C75, C76, C77, C78, C79,  C83, C84, C85, C86, C87, C88, C89,  C93, C94, C95, C96, C97, C98, C99, y63, y64, y65, y66, y67, y68, y69,  y73, y74, y75, y76, y77, y78, y79,  y83, y84, y85, y86, y87, y88, y89,  y93, y94, y95, y96, y97, y98, y99 = vars_full
    complex_vars = [ c + 1j*y for c, y in zip(vars_full[:28], vars_full[28:])]    #create 28 complex variables combining the real and imaginary parts

    povm = povm_create(complex_vars).full_POVM             # create POVM object and use that to get the POVM list
    povm_gram_matrix = POVM_gram_matrix(povm).all()         # get the gram matrix of the POVM
    residue_ortho = (1/2 )* np.sum(np.abs(povm_gram_matrix - np.eye(9)))    # calculate the residue
    return residue_ortho

In [111]:
# Trying for the new complex variabled version, with 56 variables (NOT using Lagrangian)

# takes ~ 3 mins for 10 iterations
for i in range(200, 500):
    np.random.seed(i)
    ig = np.random.rand(56)
    sol = minimize(Relation_Res, ig, method='CG', tol=1e-12)
    if i%10 == 0:
        print(i)
    if sol.fun < .0001:
        print( i, sol.fun)
        print( sol.x)

200
210
220
230
240
250
256 9.487709308792565e-05
[-0.06765182  0.75142385  0.07345708 -0.00152039  0.01613574  0.08847722
  0.2236816  -0.99985835  0.99844276  0.3956108   0.2906196   0.99609524
 -0.2216533   0.89769822  1.29019934 -0.02932978  0.6434624  -0.25155432
 -0.36271562  0.49278895  0.11850448  0.77769661  0.4958864   0.75934409
 -0.52682382  0.53400026  0.16225632  0.65440968  0.13707977 -0.87482577
 -0.2741254   0.05410329  0.92248669  1.51988402  1.05259268 -0.06495915
  0.60924219  0.87560399  1.45188224 -0.49589504  0.62984922  0.11388193
  0.48650703  0.19327132  0.94407672  0.83293457  0.09960532 -0.19215606
 -0.5690733   0.25803215  0.64642148 -0.71717566  0.41238326  0.94021283
  0.57525654 -0.81058268]
260


KeyboardInterrupt: 

In [13]:
# from the above search, seed = 210,256 gives the best result, with residue = 0.000103686...
#checking the gram matrix for this solution

# cerate complex variables

np.random.seed(256)
sol_210 = minimize(Relation_Res, np.random.rand(56), method='CG', tol=1e-12).x
print(sol_210)

complex_vars = [ c + 1j*y for c, y in zip(sol_210[:28], sol_210[28:])]    #create complex variables out of the real and imaginary parts

pp = povm_create(complex_vars).full_POVM
gram = POVM_gram_matrix(pp).all()

print('joined complex vars =' , complex_vars)


[-0.06765182  0.75142385  0.07345708 -0.00152039  0.01613574  0.08847722
  0.2236816  -0.99985835  0.99844276  0.3956108   0.2906196   0.99609524
 -0.2216533   0.89769822  1.29019934 -0.02932978  0.6434624  -0.25155432
 -0.36271562  0.49278895  0.11850448  0.77769661  0.4958864   0.75934409
 -0.52682382  0.53400026  0.16225632  0.65440968  0.13707977 -0.87482577
 -0.2741254   0.05410329  0.92248669  1.51988402  1.05259268 -0.06495915
  0.60924219  0.87560399  1.45188224 -0.49589504  0.62984922  0.11388193
  0.48650703  0.19327132  0.94407672  0.83293457  0.09960532 -0.19215606
 -0.5690733   0.25803215  0.64642148 -0.71717566  0.41238326  0.94021283
  0.57525654 -0.81058268]
joined complex vars = [(-0.06765182400465836+0.1370797747467555j), (0.7514238500025494-0.8748257731615657j), (0.073457078817426-0.27412539716844636j), (-0.0015203913243218173+0.054103292457761215j), (0.016135739643709214+0.9224866940040195j), (0.08847722456198794+1.5198840158394777j), (0.22368160149388008+1.05259268

In [29]:
print('','gram matrix rounded', np.round(gram, 4))

#visualize cleanly the gram matrix
# select elements that have abs value less than 1e-8, and replace them with 0
#pritn elements that have abs value greater than 1e-8


gram_diff = gram - np.eye(9)

print('','full gram matrix abs value')
print(np.sum(np.abs(gram_diff)))

print('sum of diagonal elements = ')
print(np.sum(np.abs(np.diag(gram_diff))))

 gram matrix rounded
2 2 1.2431961460279695e-08
2 4 2.258343008998124e-08
2 5 3.6963173000664256e-08
2 7 2.113928073012874e-07
3 5 1.1572377804171752e-05
4 2 2.258343008998124e-08
4 8 3.2314955825930184e-08
5 2 3.6963173000664256e-08
5 3 1.1572377804171752e-05
5 5 2.3560931754484926e-06
5 7 2.0772760422318687e-05
7 2 2.113928073012874e-07
7 5 2.0772760422318687e-05
7 7 0.00012132804095954519
7 8 3.5118652190947954e-07
8 4 3.2314955825930184e-08
8 7 3.5118652190947954e-07
 gram matrix abs value
0.00018971140374922683
0.00012371224551999127


The above is pretty good. Overall residue is less than .0002 which gives average angle >89.995 degrees. But we can alternatively do using Lagrange multiplies, coz it uses linear equations , so possibly better solutions.


### Lagrange method, but now using complex variables for fsolve.

In [31]:
# defining funciton for Lagrange equations - real values only. see the next cell for the complex version
def Lagrange_eqs(vars):
    # Unpacking variables, C63 to C99 and y3 to y9
    C63, C64, C65, C66, C67, C68, C69,  C73, C74, C75, C76, C77, C78, C79,  C83, C84, C85, C86, C87, C88, C89,  C93, C94, C95, C96, C97, C98, C99,  y3, y4, y5, y6, y7, y8, y9 = vars
    # global delta_five
    # delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five
    global delta_five_fixed
    delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five_fixed

    # Full set of 28 equations for Cij terms, based on previous interactions
    equations = [
        # Derivatives with respect to C63 to C69
        (1/6)*(np.conj(C63)*y3*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64)*y4*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65)*y5*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66)*y6*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67)*y7*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68)*y8*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68) + np.conj(C69)*y9*(1/6)),
        # Continue with C73 to C79
        # Derivatives with respect to C73 to C79
        (1/6)*(np.conj(C73)*y3*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74)*y4*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75)*y5*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76)*y6*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77)*y7*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78)*y8*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78) + np.conj(C79)*y9*(1/6)),
        # Derivatives with respect to C83 to C89
        (1/6)*(np.conj(C83)*y3*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84)*y4*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85)*y5*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86)*y6*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87)*y7*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88)*y8*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88) + np.conj(C89)*y9*(1/6)),
        # Derivatives with respect to C93 to C99
        (1/6)*(np.conj(C93)*y3*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94)*y4*(1/6)), 
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95)*y5*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96)*y6*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97)*y7*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98)*y8*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98) + np.conj(C99)*y9*(1/6)),
        #Now the normalization equations
        (1/6)*(abs(C63)**2 + abs(C73)**2 + abs(C83)**2 + abs(C93)**2) + delta_33 - 1,  
        (1/6)*(abs(C64)**2 + abs(C74)**2 + abs(C84)**2 + abs(C94)**2) + delta_44 - 1,
        (1/6)*(abs(C65)**2 + abs(C75)**2 + abs(C85)**2 + abs(C95)**2) + delta_55 - 1,  
        (1/6)*(abs(C66)**2 + abs(C76)**2 + abs(C86)**2 + abs(C96)**2) + delta_66 - 1,
        (1/6)*(abs(C67)**2 + abs(C77)**2 + abs(C87)**2 + abs(C97)**2) + delta_77 - 1,  
        (1/6)*(abs(C68)**2 + abs(C78)**2 + abs(C88)**2 + abs(C98)**2) + delta_88 - 1,
        (1/6)*(abs(C69)**2 + abs(C79)**2 + abs(C89)**2 + abs(C99)**2) + delta_99 - 1
    ]
    assert len(equations) == 35
    return equations




"""# Example Usage:
# ig below for np.seed = 2918 
# initial_guess =[0.98030858, 0.35567294, 0.99141023, 0.47111957, 0.34998446, 0.43261049, 0.30442865, 0.21036997, 0.39912775, 0.24321787, 0.91072068, 0.59991195, 0.39894914, 0.9238744 , 0.27633149, 0.9263308 , 0.02004467, 0.09815871, 0.03103966, 0.9009708 , 0.60346855, 0.30131413, 0.22667874, 0.99340599, 0.71874642, 0.71156703, 0.8234102 , 0.80459364, 0.77652217, 0.75734249, 0.06087287, 0.93906535, 0.92258927, 0.22224891, 0.0029041 ] 
initial_guess =[0.13645285, 0.84315108 ,0.12490685 ,0.10271676 ,0.27940578 ,0.98665734, 0.52459312, 0.41561316, 0.70258652, 0.06884098, 0.77664312, 0.84248407, 0.67034251, 0.45412921, 0.69501392, 0.69186904, 0.52676626, 0.45965484, 0.31382016, 0.70279726, 0.29404767, 0.95437514, 0.84051819, 0.87257289, 0.2450531,  0.71094717, 0.34003065, 0.7689673,  0.49019627, 0.6539388, 0.00374957, 0.24125156, 0.29499423, 0.17469819, 0.0711829 ]
print("Initial guess:", initial_guess)
# initial_guess = [0.2] * 15 + [-.2]*15 + [0.1] * 5   # Initial guess for the variables (Cij real and imaginary parts, y)
solution = fsolve(Lagrange_eqs, initial_guess)
residuals = Lagrange_eqs(solution)                   # residuals and residues sum
residuals_sum = np.sum(np.abs(residuals))

print("Solution to the system:", solution)
print("Residuals:", residuals)
print("Residuals sum:", residuals_sum)"""

'# Example Usage:\n# ig below for np.seed = 2918 \n# initial_guess =[0.98030858, 0.35567294, 0.99141023, 0.47111957, 0.34998446, 0.43261049, 0.30442865, 0.21036997, 0.39912775, 0.24321787, 0.91072068, 0.59991195, 0.39894914, 0.9238744 , 0.27633149, 0.9263308 , 0.02004467, 0.09815871, 0.03103966, 0.9009708 , 0.60346855, 0.30131413, 0.22667874, 0.99340599, 0.71874642, 0.71156703, 0.8234102 , 0.80459364, 0.77652217, 0.75734249, 0.06087287, 0.93906535, 0.92258927, 0.22224891, 0.0029041 ] \ninitial_guess =[0.13645285, 0.84315108 ,0.12490685 ,0.10271676 ,0.27940578 ,0.98665734, 0.52459312, 0.41561316, 0.70258652, 0.06884098, 0.77664312, 0.84248407, 0.67034251, 0.45412921, 0.69501392, 0.69186904, 0.52676626, 0.45965484, 0.31382016, 0.70279726, 0.29404767, 0.95437514, 0.84051819, 0.87257289, 0.2450531,  0.71094717, 0.34003065, 0.7689673,  0.49019627, 0.6539388, 0.00374957, 0.24125156, 0.29499423, 0.17469819, 0.0711829 ]\nprint("Initial guess:", initial_guess)\n# initial_guess = [0.2] * 15 + [-

In [90]:
# defining complex Lagrange equations - see previous cell for the real version
def Lagrange_eqs_complex(vars):         # vars should be a list of 70 elements, 28 real, 28 imaginary for variables, 7 real, 7 imag for lagrange multipliers
    
    complex_vars = [ c + 1j*y for c, y in zip(vars[:28], vars[28:56])]    # combining to create 28 complex numbers
    complex_y = [ c + 1j*y for c, y in zip(vars[56:63], vars[63:])]    # complex lagrange multipliers 
    # unpacking complex variables
    C63, C64, C65, C66, C67, C68, C69,  C73, C74, C75, C76, C77, C78, C79,  C83, C84, C85, C86, C87, C88, C89,  C93, C94, C95, C96, C97, C98, C99 = complex_vars
    y3, y4, y5, y6, y7, y8, y9 = complex_y

    global delta_five_fixed
    delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five_fixed

    # the 28 equations for Cij terms
    equations = [
        # Derivatives with respect to C63 to C69
        (1/6)*(np.conj(C63)*y3),
        (1/6)*(np.conj(C63) + np.conj(C64)*y4),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65)*y5),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66)*y6),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67)*y7),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68)*y8),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68) + np.conj(C69)*y9),
        # Continue with C73 to C79
        # Derivatives with respect to C73 to C79
        (1/6)*(np.conj(C73)*y3), 
        (1/6)*(np.conj(C73) + np.conj(C74)*y4), 
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75)*y5), 
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76)*y6),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77)*y7),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78)*y8),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78) + np.conj(C79)*y9),
        # Derivatives with respect to C83 to C89
        (1/6)*(np.conj(C83)*y3), 
        (1/6)*(np.conj(C83) + np.conj(C84)*y4), 
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85)*y5), 
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86)*y6),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87)*y7),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88)*y8),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88) + np.conj(C89)*y9),
        # Derivatives with respect to C93 to C99
        (1/6)*(np.conj(C93)*y3),
        (1/6)*(np.conj(C93) + np.conj(C94)*y4), 
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95)*y5),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96)*y6),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97)*y7),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98)*y8),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98) + np.conj(C99)*y9),
        #Now the normalization equations
        (1/6)*(abs(C63)**2 + abs(C73)**2 + abs(C83)**2 + abs(C93)**2) + delta_33 - 1,  
        (1/6)*(abs(C64)**2 + abs(C74)**2 + abs(C84)**2 + abs(C94)**2) + delta_44 - 1,
        (1/6)*(abs(C65)**2 + abs(C75)**2 + abs(C85)**2 + abs(C95)**2) + delta_55 - 1,  
        (1/6)*(abs(C66)**2 + abs(C76)**2 + abs(C86)**2 + abs(C96)**2) + delta_66 - 1,
        (1/6)*(abs(C67)**2 + abs(C77)**2 + abs(C87)**2 + abs(C97)**2) + delta_77 - 1,  
        (1/6)*(abs(C68)**2 + abs(C78)**2 + abs(C88)**2 + abs(C98)**2) + delta_88 - 1,
        (1/6)*(abs(C69)**2 + abs(C79)**2 + abs(C89)**2 + abs(C99)**2) + delta_99 - 1
        # Now the dummy equations for making the number of equations 63, to keep dimensions same for the fsolve function
    ]

    # Split into real and imaginary parts
    equations_real = [np.real(eq) for eq in equations]
    equations_imag = [np.imag(eq) for eq in equations]

    # Combine real and imaginary parts into a single list
    # assert len(equations) == 70
    return equations_real + equations_imag


In [91]:
# solving using fsolve for the complex variables, searching for the best seed

for i in range(0, 1000):
    np.random.seed(i)
    ig = np.random.rand(70)
    sol = fsolve(Lagrange_eqs_complex, ig)
    if i%10 == 0:
        print(i, '\n')
    if np.sum(np.abs(Lagrange_eqs_complex(sol))) < 1e-3:
        print(f'seed = {i}, res_sum = ', np.sum(np.abs(Lagrange_eqs_complex(sol))), '\n \n' , sol)


0 

10 

20 

30 

seed = 38, res_sum =  0.0002447468883855712 
 
 [-9.99406374e-01  7.57729380e-01  3.07800750e-01  8.35562384e-01
  9.48821992e-01  9.81729009e-01 -8.46741071e-01  3.41815179e-01
 -2.83557120e-01 -7.72316622e-02 -3.06916768e-01 -3.38204798e-01
 -3.36524734e-01  3.10279134e-01  2.99151489e-01 -1.00767502e+00
  8.02378003e-01 -9.40550825e-01 -7.43603048e-01 -3.50809818e-01
  9.27676436e-01  9.30559521e-01 -1.88899425e-01 -8.78350209e-01
 -3.17474946e-01 -5.75523464e-01 -8.72837795e-01  3.38637454e-01
  1.81132080e-01  6.32596353e-01 -9.38092917e-01  5.29836959e-01
  2.80819157e-01 -1.23775748e-01 -5.11996458e-01 -9.46536236e-02
 -1.95983672e-01  3.35943940e-01 -1.56773272e-01 -6.66964978e-02
  7.47084847e-02  1.50190227e-01 -1.10115674e+00  4.62890948e-01
  7.65380809e-01  5.87297565e-01  8.22433412e-01  1.05179693e+00
 -6.07258625e-01  5.24028181e-01 -1.02053645e+00  5.52534625e-01
 -9.85896237e-01 -8.61092287e-01 -5.57555564e-01  9.78795369e-01
  4.96034022e-21  6.596

In [92]:
# seed 429 is the best ~ 1e-9 sum of residuals. 3872 : res .0001, 20722:res 1e-14
good_seeds =  [148, 222, 218, 286, 307, 687]

In [106]:

np.random.seed(308)
ig = np.random.rand(70)
sol = fsolve(Lagrange_eqs_complex, ig)
res_sum = np.sum(np.abs(Lagrange_eqs_complex(sol)))

sol_coeff_c = sol[:56]
sol_coeff = [c + 1j*y for c, y in zip(sol_coeff_c[:28], sol_coeff_c[28:])]    #create complex variables out of the real and imaginary parts


print('res_sum = ', res_sum)
print('sol_coeff = ', sol_coeff)
#testing on POVM
povm20722 = povm_create(sol_coeff).full_POVM
gram20722 = POVM_gram_matrix(povm20722).all()



res_sum =  0.23350251499408406
sol_coeff =  [(-0.13939044816291898+0.4407548972023865j), (0.5756551170950136-0.12225623249178319j), (-0.5743754061787617-0.17470476174021254j), (0.7574762113699292-0.3832215024965888j), (-0.6970726511932485+0.2727634040303021j), (-0.6173438079791919-0.46572103792689784j), (-0.8147707904029199-0.0576394835824716j), (-1.1014007797788854+0.3773794602913221j), (0.941218253765609+0.5678410638018521j), (-0.05529435524446172-1.0685544340430078j), (0.034724908829604245+1.1877780135531006j), (0.3180972007323407-1.1650284143780851j), (0.5053814045975006+1.3228256343547482j), (1.1369116190662931+0.7660489976611727j), (-0.5349806013882824+0.13184263949617217j), (0.3977610150539689+0.33756876419635856j), (0.029723965993578686-0.5455658492812807j), (0.11488629978833137+0.6190227784939584j), (0.020759605498982186-0.6394339274481373j), (0.8547780789377218+0.19629917641339795j), (0.8287802927013589-0.32391694866043j), (-1.2979878276233954-0.1843542435107471j), (0.6491618

In [107]:
print('','gram matrix rounded', np.round(gram20722, 3))

 gram matrix rounded [[1.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.999 0.261 0.36  0.277 0.205 0.337 0.183]
 [0.    0.    0.261 1.001 0.547 0.035 0.252 0.376 0.253]
 [0.    0.    0.36  0.547 0.999 0.717 0.583 0.173 0.139]
 [0.    0.    0.277 0.035 0.717 1.    0.53  0.111 0.123]
 [0.    0.    0.205 0.252 0.583 0.53  0.999 0.397 0.134]
 [0.    0.    0.337 0.376 0.173 0.111 0.397 1.001 0.25 ]
 [0.    0.    0.183 0.253 0.139 0.123 0.134 0.25  1.001]]


### The lagrange method doesn't seems to work for orthogonality conditions despite fullfilling the partial derivative equations very well. So discarding this method, and proceeding with the direct that we used previously.